# Using Python to Perform Extract-Transform-Load (ETL Processing)
Modern Data Warehousing and Analytics solutions frequently use languages like Python or Scala to extract data from numerous sources, including relational database management systems, NoSQL database systems, real-time streaming endpoints and Data Lakes.  These languages can then be used to perform many types of transformation before then loading the data into a variety of destinations including file systems and data warehouses. This data can then be consumed by data scientists or business analysts.


### Prerequisites:
This notebook uses the SqlAlchemy database connectivity library to connect to MySQL databases; therefore, you must have first installed that libary into your python environment by executing the following command in a Terminal window.

- `python -m pip install sqlalchemy`

#### Import the Necessary Libraries

In [1]:
from platform import python_version
print(f"Running Python Version: {python_version()}")
import pymysql
print(f"Running PyMySQL Version: {pymysql.__version__}")
import sqlalchemy
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")


import os
import numpy
import pandas as pd
from sqlalchemy import create_engine

Running Python Version: 3.11.3
Running PyMySQL Version: 1.4.6
Running SQL Alchemy Version: 1.4.39


#### Declare & Assign Connection Variables for the MySQL Server & Databases with which You'll be Working 

In [2]:
host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "Passw0rd123"

src_dbname = "classicmodels"
dst_dbname = "classicmodels_dw2"

#### Define Functions for Getting Data From and Setting Data Into Databases

In [3]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

#### Create the New Data Warehouse database, and to Use it, Switch the Connection Context.
Clearly, you won't get very far without having a database to work with. Here we demonstrate how we can *drop* a database if it already exists, and then *create* the new **northwind_dw2** database and *use* it as the target of all subsequent operations.

In [4]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
connection.execute(f"CREATE DATABASE `{dst_dbname}`;")
connection.execute(f"USE {dst_dbname};")

connection.close()

### 1.0. Create & Populate the Dimension Tables
In any extract-transform-load (ETL) process used to populate a multi-dimensional data warehouse database it is necessary to populate the Dimension tables before attempting to populate the Fact table(s). This is because rows in the Fact table(s) will reference surrogate primary key values from the Dimension tables. If the primary key values in the Dimension tables either do not exist, or do not reflect the current state of the dimension, then the attempt to load the Fact table(s) will fail.


#### 1.1. Extract Data from the Source Database Tables
Fetch data for each dimension table (e.g., customers, employees, products, shippers) from the **classicmodels** database using the **get_dataframe()** function.

In [5]:
# Customers
sql_customers = "SELECT * FROM classicmodels.customers;"
df_customers = get_dataframe(user_id, pwd, host_name, src_dbname, sql_customers)
df_customers.head(2)

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000.0
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800.0


In [6]:
# Products
sql_products = "SELECT * FROM classicmodels.products;"
df_products = get_dataframe(user_id, pwd, host_name, src_dbname, sql_products)
df_products.head(2)

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.7
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.3


In [7]:
# ProductLInes
sql_productlines = "SELECT * FROM classicmodels.productlines;"
df_productlines = get_dataframe(user_id, pwd, host_name, src_dbname, sql_productlines)
df_productlines.head(2)

,productLine,textDescription,htmlDescription,image
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,None,None
1,Motorcycles,Our motorcycles are state of the art replicas ...,None,None


#### 1.2. Create the Date Dimension Table
At this point, we have to **execute the script from Lab 2c** that creates and populates a **Date Dimension** table.  Be certain to target this script to the new data warehouse database we just created **(classicmodels_dw2)**.  Later in this notebook we will integrate the **dim_date** table with the fact table by performing **lookup operations** to retrieve the surrogate primary keys from the **dim_date** table that correspond with each **date** typed column in the fact table (e.g., order_date).

#### 1.3. Perform Any Necessary Transformations
Pandas DataFrames enable extensive data modification capabilities. Here we will start by simply dropping features (columns) that we don't believe provide any real value to our analytics solution. Examples include columns having a high percentage of NULL values, columns having large amounts of free-text, and columns having binary large object (BLOB) data such as images or other documents. Then, we will rename the primary key column from the source (id) to serve as the business key for future lookup operations. Finally, we will *insert* a new primary key column that contains and ever-increasing numeric value.  It should be named after the entity (e.g., customer, product) followed by "**_key**" to conform with data warehouse design standards.

In [8]:
# Customers
# 1. Create a List that enumerates the names of each column you wish to remove (drop) from the Pandas DataFrame
drop_cols = ['phone','addressLine2','state']
df_customers.drop(drop_cols, axis=1, inplace=True)

# 2. Rename the "id" column to reflect the entity as it will serve as the business key for lookup operations
#df_customers.rename(columns={"id":"customer_id"}, inplace=True)

# 3. Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_customers.insert(0, "customer_key", range(1, df_customers.shape[0]+1))

# 4. Display the first 2 rows of the dataframe to validate your work
df_customers.head(2)

,customer_key,customerNumber,customerName,contactLastName,contactFirstName,addressLine1,city,postalCode,country,salesRepEmployeeNumber,creditLimit
0,1,103,Atelier graphique,Schmitt,Carine,"54, rue Royale",Nantes,44000,France,1370.0,21000.0
1,2,112,Signal Gift Stores,King,Jean,8489 Strong St.,Las Vegas,83030,USA,1166.0,71800.0


In [9]:
# Products
drop_cols = ['productDescription']
df_products.drop(drop_cols, axis=1, inplace=True)
#df_products.rename(columns={"id":"product_id"}, inplace=True)
df_products.insert(0, "product_key", range(1, df_products.shape[0]+1))
df_products.head(2)

,product_key,productCode,productName,productLine,productScale,productVendor,quantityInStock,buyPrice,MSRP
0,1,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,7933,48.81,95.7
1,2,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,7305,98.58,214.3


In [10]:
# ProductLines
drop_cols = ['htmlDescription','image']
df_productlines.drop(drop_cols, axis=1, inplace=True)
#df_productlines.rename(columns={"id":"productline_id"}, inplace=True)
df_productlines.insert(0, "productline_key", range(1, df_productlines.shape[0]+1))
df_productlines.head(2)

,productline_key,productLine,textDescription
0,1,Classic Cars,Attention car enthusiasts: Make your wildest c...
1,2,Motorcycles,Our motorcycles are state of the art replicas ...


#### 1.4. Load the Transformed DataFrames into the New Data Warehouse by Creating New Tables
Here we demonstrate how an iterable data structure can be created containing the values needed to correctly create and populate the new dimension tables. If you inspect this code listing carefully, you'll notice that it's a **list** containing a **set** *(or vector)* for each dimension table. Each **set** then contains the *table_name* we need to assign to the table, the *pandas DataFrame* we crafted to define & populate the table, and the name we need to assign to the *primary_key* column.  With this *list of sets* defined, we can then call our **set_dataframe( )** function from within a **for *loop*** to create each *dimension* table.

In [11]:
db_operation = "insert"

tables = [('dim_customers', df_customers, 'customer_key'),
          ('dim_products', df_products, 'product_key'),
          ('dim_productlines', df_productlines, 'productline_key')]

In [12]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

### 2.0. Create & Populate the Fact Table
Here we will learn two approaches to creating the *fact_orders* fact table. The first approach demonstrates that a carefully crafted SQL SELECT statement can be used to perform this task... *but what fun would that be.* Seriously though, this approach is quick and effect if you already have the query, but what if you didn't have the opportunity to view and work with the data beforehand?  What's more, you may be required to combine data from multiple sources, some of which may not be relational database management systems. Then, a simple SQL query won't do!  You would need to load the data from the various sources (e.g., database tables, CSV or JSON files, NoSQL document collections, API stream data) and then combine them into a single dataframe that you could then use to create a new database table. For this reason we'll see how we can retrieve the data, but we won't bother to use it for creating a new table... we already know how to do that using the **set_dataframe( )** function anyway.


### Implement the solution using Pandas DataFrames to craft the table
This is where we get to the point of this lab... *transforming in-memory data.*   First, we'll query the source **northwind** database to fill a *dataframe* for each of the source tables we need to create our *fact_orders* fact table; orders, orders_status, order_details and order_details_status. Then, we'll learn how to *join* those *dataframes* using the <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html">**merge( )**</a> method of the Pandas DataFrame.  We'll make any additional changes that we expect to see reflected in the *fact* table in our new MySQL database, including the addition of **foreign key references** to the dimension tables, and then we'll push the *dataframe* back to the MySQL server to create and populate the new *fact* table.

#### 2.1. Get all the data from each of the tables involved

In [13]:
sql_orders = "SELECT * FROM classicmodels.orders;"
df_orders = get_dataframe(user_id, pwd, host_name, src_dbname, sql_orders)
#df_orders.rename(columns={"id":"order_id"}, inplace=True)
df_orders.head(2)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128


In [14]:
# 1. SELECT all columns from the northwind.orders_status table to create the "df_order_status" dataframe
# 2. Rename the "id" column to "status_id"
# 3. Display the first two rows of the DataFrame to validate your work
sql_orderdetails = "SELECT * FROM classicmodels.orderdetails;"
df_orderdetails = get_dataframe(user_id, pwd, host_name, src_dbname, sql_orderdetails)
#df_orderdetails.rename(columns={"id":"orderdetails_id"}, inplace=True)
df_orderdetails.head(2)

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3
1,10100,S18_2248,50,55.09,2


#### 2.4. Join the Orders and OrderDetails DataFrames
In this step we can now easily join the *orders* and *order_details* dataframes. Since each **order** (the *left* dataframe) can have many **order details** (the *right* dataframe), we'll need to implement a **right** *outer join* **on** the *order_id* column.

In [15]:
df_fact_orders = pd.merge(df_orders, df_orderdetails, on='orderNumber', how='inner')
df_fact_orders.head(2)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,S18_1749,30,136.00,3
1,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,S18_2248,50,55.09,2


In [16]:
df_fact_orders.shape

(2996, 11)

#### 2.5. Lookup the Primary Keys from the Dimension Tables
Just as we did in **Lab 1**, we need to establish **foreign key relationships** between the newly-crafted **Fact table** and each of the **Dimension tables**.

##### 2.5.1. First, fetch the Surrogate Primary Key and the Business Key from each of the Dimension tables using the **get_dataframe()** function.

In [17]:
# Select 'customer_key' and 'customer_id' from northwind_dw2.dim_customers
sql_dim_customers = "SELECT customer_key, customerNumber FROM classicmodels_dw2.dim_customers;"
df_dim_customers = get_dataframe(user_id, pwd, host_name, src_dbname, sql_dim_customers)
df_dim_customers.head(2)

,customer_key,customerNumber
0,1,103
1,2,112


In [18]:
# Products
sql_dim_products = "SELECT product_key, productCode FROM classicmodels_dw2.dim_products;"
df_dim_products = get_dataframe(user_id, pwd, host_name, src_dbname, sql_dim_products)
df_dim_products.head(2)

,product_key,productCode
0,1,S10_1678
1,2,S10_1949


In [19]:
# ProductLines
sql_dim_productlines = "SELECT productline_key, productLine FROM classicmodels_dw2.dim_productlines;"
df_dim_productlines = get_dataframe(user_id, pwd, host_name, src_dbname, sql_dim_productlines)
df_dim_productlines.head(2)

,productline_key,productLine
0,1,Classic Cars
1,2,Motorcycles


##### 2.5.2. Next, using the Business Keys, lookup the corresponding Surrogate Primary Key values in the Dimension tables

In [20]:
# 1. Modify 'df_fact_orders' by merging it with 'df_dim_customers' on the 'customer_id' column
# 2. Drop the 'customer_id' column
# 3. Display the first 2 rows of the dataframe to validate your work


df_fact_orders = pd.merge(df_fact_orders, df_dim_customers, on='customerNumber', how='inner')
df_fact_orders.drop(['customerNumber'], axis=1, inplace=True)
df_fact_orders.head(2)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,productCode,quantityOrdered,priceEach,orderLineNumber,customer_key
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,S18_1749,30,136.00,3,86
1,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,S18_2248,50,55.09,2,86


In [21]:
# Repeat for the Products dimension
df_fact_orders = pd.merge(df_fact_orders, df_dim_products, on='productCode', how='inner')
df_fact_orders.drop(['productCode'], axis=1, inplace=True)
df_fact_orders.head(2)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,quantityOrdered,priceEach,orderLineNumber,customer_key,product_key
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,30,136.0,3,86,23
1,10379,2005-02-10,2005-02-18,2005-02-11,Shipped,None,39,156.4,2,11,23


In [22]:
# DONT RUN THIS

# Repeat for the ProductLines dimension

df_fact_orders = pd.merge(df_fact_orders, df_dim_productlines, on='productLine', how='inner')
df_fact_orders.drop(['productLine'], axis=1, inplace=True)
df_fact_orders.head(2)

KeyError: 'productLine'

##### 2.5.3. Lookup the DateKeys from the Date Dimension Table.
First, fetch the Surrogate Primary Key (date_key) and the Business Key (full_date) from the Date Dimension table using the **get_dataframe()** function. Be certain to cast the **full_date** column to the **datetime64[ns]** data type using the **.astype()** function that is native to Pandas DataFrame columns. Also, extract the **date** portion using the **.dt.date** attribute of the **datetime64[ns]** datatype.

In [24]:
sql_dim_date = "SELECT date_key, full_date FROM classicmodels_dw2.dim_date;"
df_dim_date = get_dataframe(user_id, pwd, host_name, src_dbname, sql_dim_date)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(2)

,date_key,full_date
0,20000101,2000-01-01
1,20000102,2000-01-02


Next, for each **date** typed column in the Fact table, lookup the corresponding Primary Key column. Be certain to cast each **date** column to the **datetime64[ns]** data type using the **.astype()** function that's native to Pandas DataFrame columns. Also, extract the **date** portion using the **.dt.date** attribute.

In [36]:
# Lookup the Surrogate Primary Key (date_key) that Corresponds to the "order_date" Column.
df_dim_orderDate = df_dim_date.rename(columns={"date_key" : "orderDate_key", "full_date" : "orderDate"})
df_fact_orders.orderDate = df_fact_orders.orderDate.astype('datetime64[ns]').dt.date

df_fact_orders = pd.merge(df_fact_orders, df_dim_orderDate, on='orderDate', how='left')
df_fact_orders.drop(['orderDate'], axis=1, inplace=True)
df_fact_orders.head(2)

,orderNumber,requiredDate,shippedDate,status,comments,quantityOrdered,priceEach,orderLineNumber,customer_key,product_key,orderDate_key
0,10100,2003-01-13,2003-01-10,Shipped,None,30,136.0,3,86,23,NaN
1,10379,2005-02-18,2005-02-11,Shipped,None,39,156.4,2,11,23,NaN


In [38]:
# Lookup the Surrogate Primary Key (date_key) that Corresponds to the "paid_date" Column.
df_dim_requiredDate = df_dim_date.rename(columns={"date_key" : "requiredDate_key", "full_date" : "requiredDate"})
df_fact_orders.requiredDate = df_fact_orders.requiredDate.astype('datetime64[ns]').dt.date

df_fact_orders = pd.merge(df_fact_orders, df_dim_requiredDate, on='requiredDate', how='left')
df_fact_orders.drop(['requiredDate'], axis=1, inplace=True)
df_fact_orders.head(2)

,orderNumber,shippedDate,status,comments,quantityOrdered,priceEach,orderLineNumber,customer_key,product_key,orderDate_key,requiredDate_key
0,10100,2003-01-10,Shipped,None,30,136.0,3,86,23,NaN,NaN
1,10379,2005-02-11,Shipped,None,39,156.4,2,11,23,NaN,NaN


In [40]:
# Lookup the Surrogate Primary Key (date_key) that Corresponds to the "shipped_date" Column.
df_dim_shippedDate = df_dim_date.rename(columns={"date_key" : "shippedDate_key", "full_date" : "shippedDate"})
df_fact_orders.shippedDate = df_fact_orders.shippedDate.astype('datetime64[ns]').dt.date

df_fact_orders = pd.merge(df_fact_orders, df_dim_shippedDate, on='shippedDate', how='left')
df_fact_orders.drop(['shippedDate'], axis=1, inplace=True)
df_fact_orders.head(2)

,orderNumber,status,comments,quantityOrdered,priceEach,orderLineNumber,customer_key,product_key,orderDate_key,requiredDate_key,shippedDate_key
0,10100,Shipped,None,30,136.0,3,86,23,NaN,NaN,NaN
1,10379,Shipped,None,39,156.4,2,11,23,NaN,NaN,NaN


#### 2.6. Perform any Additional Transformations
In this step we can prepare the DataFrame so that it defines exactly what we want to see created in the database.  Issues may include dropping unwanted columns, reordering the columns, and in our case, creating a new column to serve as the primary key.

In [41]:
# 1. Drop the columns of no particular interest
drop_columns = []
df_fact_orders.drop(drop_columns, axis=1, inplace=True)

# 2. Reorder the remaining columns
ordered_columns = []
df_fact_orders = df_fact_orders[ordered_columns]

# 3. Insert a new column, with an ever-incrementing numeric value, to serve as the primary key
df_fact_orders.insert(0, "fact_order_key", range(1, df_fact_orders.shape[0]+1))

# 4. Display the first 2 rows of the dataframe to validate your work
df_fact_orders.head(2)


,fact_order_key
0,1
1,2


#### 2.7. Write the DataFrame Back to the Database

In [42]:
table_name = "fact_orders"
primary_key = "fact_order_key"
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_fact_orders, table_name, primary_key, db_operation)

### 3.0. Demonstrate that the New Data Warehouse Exists and Contains the Correct Data
To demonstrate the viability of your solution, author a SQL SELECT statement that returns:
- Each Customer’s Last Name
- The total amount of the order quantity associated with each customer
- The total amount of the order unit price associated with each customer

**NOTE:** *Remember that a string typed variable whose value is contained by triple-quotes (""" ... """) can preserve multi-line formatting, and that a string variable has an intrinsic **.format()** function that accepts ordered parameters that will replace tokens (e.g., {0}) in the formatted string.*  

In [ ]:
sql_test = """
    SELECT customers.`contactLastName` AS `Customer Name`,
        SUM(orders.`quantity`) AS `Total Quantity`,
        SUM(orders.`unit_price`) AS `Total Unit Price`
    FROM `{0}`.`fact_orders` AS orders
    INNER JOIN `{0}`.`dim_customers` AS customers
    ON orders.customer_key = customers.customer_key
    GROUP BY customers.`last_name`
    ORDER BY `Total Unit Price` DESC;
""".format(dst_dbname)

df_test = get_dataframe(user_id, pwd, host_name, src_dbname, sql_test)
df_test.head()